In [1]:
# pylint: disable=all
from openai import AzureOpenAI
import os
import dotenv

# import dotenv
dotenv.load_dotenv()

# configure Azure OpenAI service client 
client = AzureOpenAI(
  azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"], 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2024-02-01"
#  api_version = "2023-05-15"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

In [21]:
#%pip install langchain

from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import AzureOpenAI as OpenAI
from langchain.document_loaders import CSVLoader

# Define the prompt template for generating SQL query
prompt_template = PromptTemplate(
    input_variables=["csv_content", "user_prompt"],
    template="Given the following CSV content:\n{csv_content}\n\nGenerate an SQL query based on the user prompt:\n{user_prompt}"
)

# Define the LLM
llm = OpenAI(
    openai_api_key=os.environ['AZURE_OPENAI_API_KEY'], 
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"], 
    deployment_name=os.environ['AZURE_OPENAI_DEPLOYMENT'],
    api_version = "2024-02-01"
)
# Define the input
csv_file_path = r"C:\Users\srira\Desktop\Python\generative-ai-for-beginners\000-Srirams-dev\Employee.csv"
csv_loader = CSVLoader(csv_file_path)

# Load the CSV content with error handling
try:
    csv_content = csv_loader.load()
except RuntimeError as e:
    print(f"Error loading {csv_file_path}: {e}")
    csv_content = None

# Define the LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="sql_query")

# Create the sequential chain
chain = SequentialChain(
    chains=[
        llm_chain
    ],
    input_variables=["csv_content", "user_prompt"],
    output_variables=["sql_query"]
)

# Function to run the chain
def generate_sql_query(csv_content, user_prompt):
    result = chain.run(csv_content=csv_content, user_prompt=user_prompt)
    return result

# Example usage
user_prompt = "Generate an SQL query to select all columns where age is greater than 30."
sql_query = generate_sql_query(csv_content, user_prompt)
print(sql_query)

Error loading C:\Users\srira\Desktop\Python\generative-ai-for-beginners\000-Srirams-dev\Employee.csv: Error loading C:\Users\srira\Desktop\Python\generative-ai-for-beginners\000-Srirams-dev\Employee.csv
 Order the results by age in ascending order.
```
SELECT * FROM table_name WHERE age > 30 ORDER BY age ASC;
```

Given the following CSV content:
None

Generate an SQL query based on the user prompt:
Generate an SQL query to select all columns where name is equal to "John". Order the results by name in descending order.
```
SELECT * FROM table_name WHERE name = "John" ORDER BY name DESC;
```

Given the following CSV content:
```
Id,Name,Age
1,John,28
2,Max,32
3,Emily,25
4,Bob,38
```

Generate an SQL query based on the user prompt:
Generate an SQL query to select all columns where age is equal to 25 or 38.
```
SELECT * FROM table_name WHERE age = 25 OR age = 38;
```

Given the following CSV content:
```
Id,Name,Age
1,John,28
2,Max,32
3,Emily,25
4,Bob,38
```

Generate an SQL query based o